*Copyright 2019 StarkWare Industries Ltd.<br> Licensed under the Apache License, Version 2.0 (the "License"). You may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.starkware.co/open-source-license/ <br> Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.*

# Part 2: Constraints

([Part 2 Presentation](https://starkware.co/stark101-pdfs//STARK101-Part2.pdf) in PDF format.)

In this part, we are going to create a set of constraints over the trace `a`. <br>The constraints will be expressions in the trace's cells that are polynomials (rather than [rational functions](https://en.wikipedia.org/wiki/Rational_function)) if and only if the trace represents a valid computation of the FibonacciSq. <br>
<br>
We will get there in three steps:
1. Start by specifying the constraints we care about (the **FibonacciSq constraints**).
2. Translate the FibonacciSq constraints into **polynomial constraints**.
3. Translate those into **rational functions** that represent polynomials if and only if the original constraints hold. 

## Step 1 - FibonacciSq Constraints
For `a` to be a correct trace of a FibonacciSq sequence that proves our claim:
1. The first element has to be 1, namely $a[0] = 1$.
2. The last element has to be 2338775057, namely $a[1022] = 2338775057$.
3. The FibonacciSq rule must apply, that is - for every $i<1021$, $a[i+2]=a[i+1]^2+a[i]^2$.

## Step 2 - Polynomial Constraints
Recall that `f` is a polynomial over the trace domain, that evaluates exactly to `a` over $G \setminus \{g^{1023}\}$ where $G=\{g^i : 0\leq i\leq 1023\}$ is the "small" group generated by $g$.<br>

We now rewrite the above three constraints in a form of polynomial constraints over `f`:
1. $a[0] = 1$ is translated to the polynomial $f(x) - 1$, which evalutes to 0 for $x = g^0$ (note that $g^0$ is $1$). <br>
2. $a[1022] = 2338775057$ is translated to the polynomial $f(x) - 2338775057$, which evalutes to 0 for $x = g^{1022}$. <br>
3. $a[i+2]=a[i+1]^2+a[i]^2$ for every $i<1021$ is translated to the polynomial $f(g^2 \cdot x) - (f(g \cdot x))^2 - (f(x))^2$, which evaluates to 0 for $x \in G \backslash \{g^{1021}, g^{1022}, g^{1023}\}$. <br><br>

### Hands on
First, since this is a separate notebook from Part 1, let's run the following piece of code to have all the variables here with their correct values. Note that it may take up to 30 seconds, since it reruns the polynomial interpolation.

In [ ]:
from channel import Channel
from field import FieldElement
from merkle import MerkleTree
from polynomial import interpolate_poly, X, prod
from tutorial_sessions import part1

a, g, G, h, H, eval_domain, f, f_eval, f_merkle, channel = part1()
print('Success!')

You will obtain each of the three constraints as a quotient of two polynomials, making sure the remainder is the zero polynomial. 
<br><br>

## Step 3 - Rational Functions (That are in Fact Polynomials)

Each of the constraints above is represented by a polynomial $u(x)$ that supposedly evaluates to $0$ on certain elements of the group $G$. That is, for some $x_0, \ldots, x_k \in G$, we claim that

$$u(x_0) = \ldots = u(x_k) = 0$$

(note that for the first two constaints, $k=0$ because they only refer to one point and for the third $k=1021$).

This is equivalent to saying that $u(x)$ is divisible, as a polynomial, by all of $\{(x-x_i)\}_{i=0}^k$, or, equivalently, by

$$\prod_{i=0}^k (x-x_i)$$

Therefore, each of the three constraints above can be written as a rational function of the form:

$$\frac{u(x)}{\prod_{i=0}^k (x-x_i)}$$

for the corresponding $u(x)$ and $\{x_i\}_{i=0}^k$. In this step we will construct these three rational functions and show that they are indeed polynomials.

## The First Constraint:

In the first constraint, $f(x) - 1$ and $\{x_i\} = \{1\}$.

We will now construct the **polynomial** $p_0(x)=\frac{f(x) - 1}{x - 1}$, making sure that $f(x) - 1$ is indeed divisible by $(x-1)$.

In [ ]:
# First constraint. Construct numer0 and denom0.
numer0 = 'YOUR_CODE_HERE'
denom0 = 'YOUR_CODE_HERE'

Solution:

In [ ]:
numer0 = f - 1
denom0 = X - 1

Convince yourself that $f(x) - 1$ vanishes at $x=1$ by making sure that evaluating this polynomial at $1$ yields $0$:

In [ ]:
'YOUR_CODE_HERE'

The fact that $f(x) - 1$ has a root at $1$ implies that it is divisible by $(x - 1)$.
Run the following cell to convince yourself that the remainder of `numer0` modulo `denom0` is $0$, and therefore division indeed yields a polynomial:

In [ ]:
numer0 % denom0

Run the following cell to construct `p0`, the polynomial representing the first constraint, by dividing `numer0` by `denom0`:

In [ ]:
p0 = numer0 / denom0

Run test:

In [ ]:
assert p0(2718) == 2509888982
print('Success!')

## The Second Constraint

Construct the polynomial `p1` representing the  second constraint, $p_1(x)= \frac{f(x) - 2338775057}{x - g^{1022}}$, similarly: <br>

In [ ]:
# Second constraint.
p1 = 'YOUR_CODE_HERE'

Solution:

In [ ]:
numer1 = f - 2338775057
denom1 = X - g**1022
p1 = numer1 / denom1

Run test:

In [ ]:
assert p1(5772) == 232961446
print('Success!')

## The Third Constraint - Succinctness

The last constraint's rational function is slightly more complicated: <br>


$$p_2(x) = \frac{f(g^2 \cdot x) - (f(g \cdot x))^2 - (f(x))^2}{\prod\limits_{i=0}^{1020} (x-g^i)}$$

whose denominator can be rewritten, so that the entire expression is easier to compute:<br>

$$\frac{f(g^2 \cdot x) - (f(g \cdot x))^2 - (f(x))^2}{\frac{x^{1024} - 1}{(x-g^{1021})(x-g^{1022})(x-g^{1023})}}$$ <br>

This follows from the equality

$$\prod\limits_{i=0}^{1023} (x-g^i) = x^{1024} - 1$$

Convince yourself of this equality using the function `prod` that takes a list and computes its product:

In [ ]:
# Construct a list `lst` of the linear terms (x-g**i):
lst = ['YOUR_CODE_HERE']
# Compute the product of `lst` and see that it is indeed the succinct polynomial x**1024 - 1
prod(lst)

Solution:

In [ ]:
lst = [(X - g**i) for i in range(1024)]
prod(lst)

For more information, see our blog post titled [Arithmetization II](https://medium.com/starkware/arithmetization-ii-403c3b3f4355).

Let's pause for a moment, and look at a simple example on how polynomials are composed. After that we will generate the third constraint.

### Composing Polynomials (a detour)

Create the two polynomials $q(x) = 2x^2 +1$, $r(x) = x - 3$:

In [ ]:
q = 2*X ** 2 + 1
r = X - 3

Composing $q$ on $r$ yields a new polynomial:<br>
$q(r(x)) = 2(x-3)^2 + 1 = 2x^2-12x+19$
<br>
Run the following cell to create a third polynomial `cmp` by composing `q` on `r` and convince yourself that `cmp` is indeed the composition of `q` and `r`:

In [ ]:
cmp = q(r)
cmp

### Back to Polynomial Constraints
Construct the third constraint `p2` in a similar manner to the construction of `p0` and `p1`, using polynomial composition. Along the way, verify that $g^{1020}$ is a root of the **numerator** while $g^{1021}$ is not.

In [ ]:
p2 = 'YOUR_CODE_HERE'

Solution:

In [ ]:
numer2 = f(g**2 * X) - f(g * X)**2 - f**2
print("Numerator at g^1020 is", numer2(g**1020))
print("Numerator at g^1021 is", numer2(g**1021))
denom2 = (X**1024 - 1) / ((X - g**1021) * (X - g**1022) * (X - g**1023))

p2 = numer2 / denom2

Run test:

In [ ]:
assert p2.degree() == 1023, f'The degree of the third constraint is {p2.degree()} when it should be 1023.'
assert p2(31415) == 2090051528
print('Success!')

Run the following cell to observe the degrees of the constraint polynomials `p0`, `p1` and `p2`, all less than $1024$. This will be important in the next part.

In [ ]:
print('deg p0 =', p0.degree())
print('deg p1 =', p1.degree())
print('deg p2 =', p2.degree())

### Step 4 - Composition Polynomial
Recall that we're translating a problem of checking the validity of three polynomial constraints to checking that each of the rational functions $p_0, p_1, p_2$ are polynomials. <br>

Our protocol uses an algorithm called [FRI](https://eccc.weizmann.ac.il/report/2017/134/) to do so, which will be discussed in the next part. <br>
In order for the proof to be succinct (short), we prefer to work with just one rational function instead of three. For that, we take a random linear combination of $p_0, p_1, p_2$ called the **compostion polynomial** (CP for short):

$$CP(x) = \alpha_0 \cdot p_0(x) + \alpha_1 \cdot p_1(x) + \alpha_2 \cdot  p_2(x)$$ <br>

where $\alpha_0, \alpha_1, \alpha_2 $ are random field elements obtained from the verifier, or in our case - from the channel.

Proving that (the rational function) $CP$ is a polynomial guarantess, with high probability, that each of $p_0$, $p_1$, $p_2$ are themselves polynomials.

In the next part, you will generate a proof for an equivalent fact. But first, let's create `CP` using `Channel.receive_random_field_element` to obtain $\alpha_i$: <br> 
   

In [ ]:
# Note that alpha0, alpha1, alpha2 have to be drawn from the channel in this order.
def get_CP(channel):
    return 'YOUR_CODE_HERE'

Solution:

In [ ]:
def get_CP(channel):
    alpha0 = channel.receive_random_field_element()
    alpha1 = channel.receive_random_field_element()
    alpha2 = channel.receive_random_field_element()
    return alpha0*p0 + alpha1*p1 + alpha2*p2

Run test:

In [ ]:
test_channel = Channel()
CP_test = get_CP(test_channel)
assert CP_test.degree() == 1023, f'The degree of cp is {CP_test.degree()} when it should be 1023.'
assert CP_test(2439804) == 838767343, f'cp(2439804) = {CP_test(2439804)}, when it should be 838767343'
print('Success!')

### Commit on the Composition Polynomial
Lastly, we evaluate $cp$ over the evaluation domain (`eval_domain`), build a Merkle tree on top of that and send its root over the channel. This is similar to commiting on the LDE trace, as we did at the end of part 1.

In [ ]:
# Fix this. CP_eval is the evaluation of CP on all the points in domain. For a hint - look at "Evaluate on a Coset"
# on part 1.
def CP_eval(channel):
    CP = get_CP(channel)
    return 'YOUR_CODE_HERE'

Solution:

In [ ]:
def CP_eval(channel):
    CP = get_CP(channel)
    return [CP(d) for d in eval_domain]

Construct a Merkle Tree over the evaluation and send its root over the channel.

In [ ]:
channel = Channel()
CP_merkle = MerkleTree(['YOUR_CODE_HERE']) # Fix this line
channel.send(CP_merkle.root)

Solution:

In [ ]:
channel = Channel()
CP_merkle = MerkleTree(CP_eval(channel))
channel.send(CP_merkle.root)

Test your code:

In [ ]:
assert CP_merkle.root == 'a8c87ef9764af3fa005a1a2cf3ec8db50e754ccb655be7597ead15ed4a9110f1', 'Merkle tree root is wrong.'
print('Success!')